Get the average temporal correlation values of all the spontaneous activities in one exp for 2 cells

In [47]:
# ec_id = 511510650
eid = 501836392 #501886692 three_session_B 501836392 three_session_A 501717543 three_session_C
c1 = 517419703 
c2 = 517419533 
use_events = True

In [48]:
import os
drive_path = '/media/stephen/Brain20191/allen-brain-observatory/visual-coding-2p/'
if not os.path.isdir(drive_path):
    drive_path = '/media/alihamodi/Brain2019/allen-brain-observatory/visual-coding-2p/'
    
import warnings;
warnings.simplefilter('ignore');
import os
import pandas as pd
from allensdk.core.brain_observatory_cache import BrainObservatoryCache
import allensdk.brain_observatory.stimulus_info as stim_info
from utils.data_wrappers import *
from utils.polardata import *
from utils.polarplot import *
manifest_file = os.path.join(drive_path,'manifest.json')
boc = BrainObservatoryCache(manifest_file=manifest_file)

In [49]:
import math
data_set = boc.get_ophys_experiment_data(eid)

try: 
    cidxs = data_set.get_cell_specimen_indices([c1, c2])
except Exception as inst:
    print(inst)

if use_events:
  events = boc.get_ophys_experiment_events(ophys_experiment_id=eid)
  cidx1 = cidxs[0]
  cidx2 = cidxs[1]
  events1 = events[cidx1,:]
  events2 = events[cidx2,:]
else: 
  timestamps, dff = data_set.get_dff_traces(cell_specimen_ids=[c1])
  events1 = dff[0,:]
  timestamps, dff = data_set.get_dff_traces(cell_specimen_ids=[c2])
  events2 = dff[1,:]
"""
'natural_movie_one',
'natural_movie_three',
'natural_movie_two',
'spontaneous'
"""
stim_table = data_set.get_stimulus_table('natural_movie_one') 
# spontaneous
# Each item is a temporal correlation of a single spontaneous presentation
temp_corr_lists = []
# Each row in stim table for the movie is a frame of a particular trial
# Trial number = repeat. 
for trial_i in range(stim_table.repeat.max() + 1):
    start = stim_table[stim_table.repeat==trial_i].start.min()
    end = stim_table[stim_table.repeat==trial_i].start.max()
    ts1 = events1[start:end]
    ts2 = events2[start:end]
    temp_corr, p_value = pearsonr(ts1, ts2)
    if math.isnan(temp_corr):
        continue
    temp_corr_lists.append(temp_corr)
    print(trial_i, temp_corr)
if len(temp_corr_lists) == 0:
    return None
return np.mean(temp_corr_lists)

0 -0.0036791408933685484
1 -0.0030731212399439465
2 -0.003121774462440617
3 -0.002695464221437613
4 -0.006340035571979338
5 -0.004020608482217043
6 -0.0011074197120708765
7 -0.004556807560653523
9 -0.003036713312617691


SyntaxError: 'return' outside function (<ipython-input-49-4b9458c65099>, line 43)

In [7]:
from utils.data_wrappers import *
avg_temp_corr_one_exp(boc, eid, c1, c2)

-0.0019509815338686607

In [16]:

stim_table = data_set.get_stimulus_table('natural_movie_one')

In [42]:
stim_table[stim_table.repeat==4]


,frame,start,end,repeat
3600,0,42385,42385,4
3601,1,42386,42386,4
3602,2,42387,42387,4
3603,3,42388,42388,4
3604,4,42389,42389,4
3605,5,42390,42391,4
3606,6,42391,42392,4
3607,7,42392,42393,4
3608,8,42393,42394,4
3609,9,42394,42395,4
